In [10]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


d:\software\sayed\anaconda\envs\torch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
pattern3 = r'P+'  # Define the regex pattern to match consecutive 'P's
def process3(x):
    p = re.sub(pattern3, "", x)  # Use re.sub to replace the pattern with an empty string
    return p  # Return the processed string


In [68]:
# Apply the process1 function to each element in the target_text column
new.target_text = new.target_text.apply(lambda x: process1(x))


In [76]:
predict.head()

,target,prediction
0,2796 west golden willow drive,189 steth werere re roure
1,9734719887,aranere-199
2,4977236992,989-111-6662
3,reallyloud.co.uksimaii,delen rereron
4,kkaicd1.pixnet.net,arisalallange


In [83]:
from transformers.modeling_tf_utils import get_initializer
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import os
import pandas as pd


In [84]:
# Set the number of threads for intra-op and inter-op parallelism
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

# Check if a pretrained model exists, otherwise download it
if os.path.exists("tf_gpt2_keras_lora"):
    print("Model exists")
    # Use the pretrained model if it exists
    model = TFGPT2LMHeadModel.from_pretrained("tf_gpt2_keras_lora")
else:
    print("Downloading model")
    # Download and use the gpt2 pretrained model
    model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [22]:
data = pd.read_csv("D:/data/output.csv")

In [23]:
data.head()

,prediction,target_text
0,1795 westh st rerorore,2796 west golden willow drive
1,+39-313-5199,9734719887
2,998-121-1662,4977236992
3,elle cororerin,reallyloud.co.uksimaii
4,arilalallanges,kkaicd1.pixnet.net


In [85]:
# Assuming 'data' is the collective dataset containing predictions and targets
# Replace 'collective' with the correct path and file name if needed

# Extract the 'prediction' column values as a list
inputs = data["prediction"].tolist()

# Extract the 'target' column values as a list
targets = data["target"].tolist()


In [86]:
# Import required libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
#import distance 

# Initialize empty lists to store input and target encodings
input_ids = []
target_ids = []

# Loop through inputs and targets
for inp, tgt in zip(inputs, targets):
    # Encode input and target sequences using the tokenizer
    input_encoding = tokenizer.encode(inp, add_special_tokens=True, max_length=1024, truncation=True)
    target_encoding = tokenizer.encode(tgt, add_special_tokens=True, max_length=1024, truncation=True)
    
    # Append encoded input and target sequences to lists
    input_ids.append(input_encoding)
    target_ids.append(target_encoding)

# Split the data into train and test sets
input_train, input_test, target_train, target_test = train_test_split(input_ids, target_ids, test_size=0.2, random_state=42)

# Define some parameters for training
batch_size = 2
num_epochs = 3
learning_rate = 5e-5

# Define the optimizer and loss function for the model training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)



In [87]:
import distance

In [88]:
# Calculate Levenshtein distance-based accuracy
def calculate_accuracy(predictions, targets):
    accuracies = []  # Create an empty list to store individual accuracies
    for pred, target in zip(predictions, targets):  # Loop through predicted and target sequences
        pred_text = tokenizer.decode(pred, skip_special_tokens=True)  # Convert predicted token IDs to text
        target_text = tokenizer.decode(target, skip_special_tokens=True)  # Convert target token IDs to text
        # Calculate Levenshtein distance-based accuracy for this pair of sequences
        accuracy = 1 - (distance.levenshtein(pred_text, target_text) / max(len(pred_text), len(target_text)))
        accuracies.append(accuracy)  # Add accuracy to the list
    return np.mean(accuracies)  # Calculate and return the mean accuracy across all pairs


In [38]:
# Define some params
batch_size = 2
num_epochs = 3
learning_rate = 5e-5

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Fine-tune the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i in range(0, len(input_ids), batch_size):
        batch_inputs = input_ids[i:i+batch_size]
        batch_targets = target_ids[i:i+batch_size]

        # Find the maximum sequence length in the batch
        max_length = max(max(len(seq) for seq in batch_inputs), max(len(seq) for seq in batch_targets)
        
        # Pad the input and target sequences to the same maximum length
        batch_inputs = tf.keras.preprocessing.sequence.pad_sequences(batch_inputs, padding="post", maxlen=max_length)
        batch_targets = tf.keras.preprocessing.sequence.pad_sequences(batch_targets, padding="post", maxlen=max_length)

        # Compute the predictions and loss
        with tf.GradientTape() as tape:
            logits = model(batch_inputs)[0]
            loss = loss_fn(batch_targets, logits)

        # Compute the gradients and update the parameters
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Print the loss every 10 batches
        if i % (10 * batch_size) == 0:
            print(f"Batch {i}/{len(input_ids)} - loss: {loss:.4f}")


Epoch 1/3
Batch 0/512 - loss: 9.0748
Batch 20/512 - loss: 6.6302
Batch 40/512 - loss: 7.4509
Batch 60/512 - loss: 7.2791
Batch 80/512 - loss: 5.7221
Batch 100/512 - loss: 5.0603
Batch 120/512 - loss: 4.6360
Batch 140/512 - loss: 3.9326
Batch 160/512 - loss: 6.1096
Batch 180/512 - loss: 2.7365
Batch 200/512 - loss: 2.8067
Batch 220/512 - loss: 4.6523
Batch 240/512 - loss: 2.2090
Batch 260/512 - loss: 5.9496
Batch 280/512 - loss: 4.2090
Batch 300/512 - loss: 5.5323
Batch 320/512 - loss: 4.2686
Batch 340/512 - loss: 3.3355
Batch 360/512 - loss: 4.0083
Batch 380/512 - loss: 3.0165
Batch 400/512 - loss: 1.6922
Batch 420/512 - loss: 2.5310
Batch 440/512 - loss: 3.7592
Batch 460/512 - loss: 3.4074
Batch 480/512 - loss: 1.6323
Batch 500/512 - loss: 0.4391
Epoch 2/3
Batch 0/512 - loss: 4.8635
Batch 20/512 - loss: 4.7457
Batch 40/512 - loss: 5.2103
Batch 60/512 - loss: 5.2140
Batch 80/512 - loss: 3.7198
Batch 100/512 - loss: 2.9074
Batch 120/512 - loss: 2.9727
Batch 140/512 - loss: 1.5464
Batch 

In [94]:
# Example of generating text using the loaded model
# Input text to generate from
input_text = "are you donet comonn"

# Encode the input text using the tokenizer
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=1024, truncation=True)

# Generate text with the loaded model
output = model.generate(
    tf.constant([input_ids]),  # Provide the encoded input_ids
    max_length=100,             # Maximum length of the generated text
    do_sample=True,             # Allow for sampling
    top_k=50,                   # Top-k tokens to consider during sampling
    top_p=0.95,                 # Top-p (nucleus) sampling
    temperature=0.9,            # Sampling temperature
    pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id to eos_token_id
    attention_mask=tf.constant([[1] * len(input_ids)]),  # Pass attention_mask
)

# Decode the generated output to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the input and generated text
print(f"Input: {input_text}")
print(f"Generated text: {generated_text}")


Input: are you donet comonn
Generated text: are you donet comonn!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
# Split the dataset into training and testing sets
input_train, input_test, target_train, target_test = train_test_split(input_ids, target_ids, test_size=0.2, random_state=42)

# Function to calculate Levenshtein distance-based accuracy
def calculate_accuracy(predictions, targets):
    accuracies = []
    for pred, target in zip(predictions, targets):
        pred_text = tokenizer.decode(pred, skip_special_tokens=True)
        target_text = tokenizer.decode(target, skip_special_tokens=True)
        accuracy = 1 - (distance.levenshtein(pred_text, target_text) / max(len(pred_text), len(target_text)))
        accuracies.append(accuracy)
    return np.mean(accuracies)

# Generate predictions for the test set
test_predictions = loaded_model.generate(
    tf.constant(input_test),
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id,
    attention_mask=tf.constant([[1] * len(seq) for seq in input_test])
)

# Calculate and print the accuracy
accuracy = calculate_accuracy(test_predictions, target_test)
print("Levenshtein Distance-based Accuracy:", accuracy)
